In [1]:
import numpy as np

In [12]:
x = np.array([1, 2, 4, -9, 5, 3])
h = np.array([1/3, 1/3, 1/3])

In [13]:
def convolve_direct(x, h):
    """
    Tích chập rời rạc 'full' (độ dài = len(x)+len(h)-1).
    Cài đặt trực tiếp (naive) O(N*M).
    x, h: 1D array-like
    trả về: numpy array (float)
    """
    x = np.asarray(x, dtype=float)
    h = np.asarray(h, dtype=float)
    nx = x.size
    nh = h.size
    ny = nx + nh - 1
    y = np.zeros(ny, dtype=float)
    for i in range(nx):
        for j in range(nh):
            y[i + j] += x[i] * h[j]
    return y

convolve_direct_val = convolve_direct(x, h)
print(convolve_direct_val)

[ 3.33333333e-01  1.00000000e+00  2.33333333e+00 -1.00000000e+00
 -2.22044605e-16 -3.33333333e-01  2.66666667e+00  1.00000000e+00]


In [14]:
def convolve_same(x, h):
    """
    Tương tự numpy.convolve(mode='same'): trả về mảng có cùng độ dài như x
    (căn giữa dựa trên h).
    """
    x = np.asarray(x, dtype=float)
    full = convolve_direct(x, h)
    nh = np.asarray(h).size
    start = (nh - 1) // 2
    return full[start:start + x.size]

convolve_same_val = convolve_direct(x, h)
print(convolve_same_val)

[ 3.33333333e-01  1.00000000e+00  2.33333333e+00 -1.00000000e+00
 -2.22044605e-16 -3.33333333e-01  2.66666667e+00  1.00000000e+00]


In [9]:
def convolve_valid(x, h):
    """
    Trả về 'valid' mode: những vị trí toàn phần h nằm trong x
    (độ dài = max(0, len(x)-len(h)+1))
    """
    x = np.asarray(x, dtype=float)
    h = np.asarray(h, dtype=float)
    nx, nh = x.size, h.size
    if nh > nx:
        return np.array([])
    full = convolve_direct(x, h)
    start = nh - 1
    end = start + (nx - nh + 1)
    return full[start:end]

convolve_valid_val = convolve_valid(x, h)
print(convolve_valid_val)

[ 2.33333333e+00 -1.00000000e+00 -2.22044605e-16 -3.33333333e-01]


In [15]:
def next_pow2(n):
    """Next power of two >= n"""
    return 1 << ((n - 1).bit_length())

# next_pow2_val = next_pow2()

In [11]:
def convolve_fft(x, h):
    """
    Tích chập bằng FFT (độ dài 'full').
    Phù hợp cho tín hiệu dài (complexity ~ O(N log N)).
    """
    x = np.asarray(x, dtype=float)
    h = np.asarray(h, dtype=float)
    nx, nh = x.size, h.size
    n = nx + nh - 1
    N = next_pow2(n)  # zero-pad lên 2^k cho nhanh
    X = np.fft.fft(x, N)
    H = np.fft.fft(h, N)
    Y = X * H
    y = np.fft.ifft(Y)[:n].real  # loại bỏ phần ảo do sai số số học
    return y

convolve_fft_val = convolve_fft(x, h)
print(convolve_fft_val)

[ 0.33333333  1.          2.33333333 -1.          0.         -0.33333333
  2.66666667  1.        ]


In [16]:
y_direct = convolve_direct(x, h)
y_fft = convolve_fft(x, h)
y_same = convolve_same(x, h)
y_valid = convolve_valid(x, h)
y_np = np.convolve(x, h)

In [18]:
print("x =", x)
print("h =", h)
print("\nKết quả (full convolution):")
print("convolve_direct:", y_direct)
print("\nconvolve_fft   :", np.round(y_fft, 8))
print("\nnumpy.convolve :", y_np)

x = [ 1  2  4 -9  5  3]
h = [0.33333333 0.33333333 0.33333333]

Kết quả (full convolution):
convolve_direct: [ 3.33333333e-01  1.00000000e+00  2.33333333e+00 -1.00000000e+00
 -2.22044605e-16 -3.33333333e-01  2.66666667e+00  1.00000000e+00]

convolve_fft   : [ 0.33333333  1.          2.33333333 -1.          0.         -0.33333333
  2.66666667  1.        ]

numpy.convolve : [ 3.33333333e-01  1.00000000e+00  2.33333333e+00 -1.00000000e+00
 -2.22044605e-16 -3.33333333e-01  2.66666667e+00  1.00000000e+00]


In [19]:
print("\nSo sánh direct == numpy:", np.allclose(y_direct, y_np))
print("So sánh fft == numpy   :", np.allclose(y_fft, y_np))


So sánh direct == numpy: True
So sánh fft == numpy   : True


In [21]:
print("\nMode 'same' (cùng độ dài như x):", y_same)
print("\nMode 'valid':", y_valid)


Mode 'same' (cùng độ dài như x): [ 1.00000000e+00  2.33333333e+00 -1.00000000e+00 -2.22044605e-16
 -3.33333333e-01  2.66666667e+00]

Mode 'valid': [ 2.33333333e+00 -1.00000000e+00 -2.22044605e-16 -3.33333333e-01]


In [22]:
print("\nĐộ dài x:", x.size, "Độ dài h:", h.size, "Độ dài full y:", y_direct.size)
print("Độ phức tạp direct: O(N*M) =", x.size, "*", h.size)
print("Độ phức tạp FFT: ~ O(L log L) với L =", next_pow2(x.size + h.size - 1))


Độ dài x: 6 Độ dài h: 3 Độ dài full y: 8
Độ phức tạp direct: O(N*M) = 6 * 3
Độ phức tạp FFT: ~ O(L log L) với L = 8
